In [30]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns
from sklearn.model_selection import KFold

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from joblib import dump, load
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [31]:
from sklearn.preprocessing import QuantileTransformer


In [32]:
train_features = pd.read_csv('D:\\Dataset\\MOA\\train_features.csv')
train_targets_scored = pd.read_csv('D:\\Dataset\\MOA\\train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('D:\\Dataset\\MOA\\train_targets_nonscored.csv')

test_features = pd.read_csv('D:\\Dataset\\MOA\\test_features.csv')
sample_submission = pd.read_csv('D:\\Dataset\\MOA\\sample_submission.csv')

In [33]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [34]:
transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
transformer.fit(train_features.loc[:, GENES + CELLS])
train_features[GENES + CELLS] = transformer.transform(train_features.loc[:, GENES + CELLS])

dump(transformer, 'rank_1838_5fold_sub.bin', compress=True)


test_features[GENES + CELLS] = transformer.transform(test_features.loc[:, GENES + CELLS])

In [35]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [36]:
# GENES
n_comp = 600  #<--Update

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
pca_genes = PCA(n_components=n_comp, random_state=42)
data2 = pca_genes.fit_transform(data[GENES])

dump(pca_genes, '1838_5fold_genes_sub.bin', compress=True)
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [37]:
#CELLS
n_comp = 50  #<--Update

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
pca_cells = PCA(n_components=n_comp, random_state=42)
data2 = pca_cells.fit_transform(data[CELLS])

dump(pca_cells, '1838_5fold_cells_sub.bin', compress=True)


train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [38]:
from sklearn.feature_selection import VarianceThreshold


var_thresh = VarianceThreshold(0.8)  #<-- Update
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

dump(var_thresh, 'var_1838_5fold_sub.bin', compress=True)

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

(23814, 1040)

In [39]:
#train = train_features.merge(train_targets_scored, on='sig_id')
drug = pd.read_csv('D://Dataset//MOA//train_drug.csv')

#train = train.merge(drug, on='sig_id')

train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(drug, on='sig_id')

train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]


In [40]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [41]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [42]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

def create_folds(num_starts, num_splits, trn):

    folds = []
    scored = trn.copy()
    #targets_ = scored.loc[:, train_targets_scored.columns].columns[1:].tolist()
    #train_cols = train_features.columns.tolist() + ['fold','drug_id']
    #train_cols = [col for col in train_cols if col!='cp_type']
    # LOCATE DRUGS
    vc = scored.drug_id.value_counts()
    vc1 = vc.loc[vc <= 18].index.sort_values()
    vc2 = vc.loc[vc > 18].index.sort_values()
    
    for seed in range(num_starts):
    
        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}; dct2 = {}
        skf = KFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.groupby('drug_id')[target_cols].mean().loc[vc1]
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
            dd = {k:fold for k in tmp.index[idxV].values}
            dct1.update(dd)
    
        # STRATIFY DRUGS MORE THAN 18X
        skf = KFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop = True)
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
            dd = {k:fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)
    
        # ASSIGN FOLDS
        scored['fold'] = scored.drug_id.map(dct1)
        scored.loc[scored.fold.isna(),'fold'] =\
            scored.loc[scored.fold.isna(),'sig_id'].map(dct2)
        scored.fold = scored.fold.astype('int8')
        folds.append(scored.fold.values)
        #train_cols = train_feats.columns.tolist() + ['fold','drug_id']
        #train_feats_main = train_feats_.merge(scored, on='sig_id', how='left')
        #train_feats_main['fold'] = train_feats_main['fold'].astype(int)
        
    return scored

In [43]:
folds= create_folds(1, 5, train)

In [44]:
folds

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,...,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,drug_id,fold
0,id_000644bb2,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,-1.017473,-1.364787,...,0,0,0,0,0,0,0,0,b68db1d53,4
1,id_000779bfc,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,0.686517,0.313396,...,0,0,0,0,0,0,0,0,df89a8e5a,1
2,id_000a6266a,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,1.492493,0.235577,...,0,0,0,0,0,0,0,0,18bb41b2c,0
3,id_0015fd391,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,-0.858153,-2.288417,...,0,0,0,0,0,0,0,0,8c7f86626,1
4,id_001626bd3,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,-0.869555,-0.375501,...,0,0,0,0,0,0,0,0,7cbed3131,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,72,D1,0.237856,-1.228203,0.218376,-0.365976,-0.330177,0.569243,-0.150978,...,0,0,0,0,0,0,0,0,6c3a459be,2
21944,id_fffb1ceed,24,D2,0.209361,-0.022389,-0.235888,-0.796989,-0.674009,0.919312,0.735603,...,0,0,0,0,0,0,0,0,df1d0a5a1,4
21945,id_fffb70c0c,24,D2,-1.911021,0.587228,-0.588417,1.296405,-1.002640,0.850589,-0.304313,...,0,0,0,0,0,0,0,0,ecf3b6b74,0
21946,id_fffcb9e7c,24,D1,0.816407,0.417618,0.431631,0.300617,1.070346,-0.024189,0.048942,...,0,0,0,0,0,0,0,0,8b87a7a83,0


In [45]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

(21948, 1246)
(21948, 1247)
(3624, 1039)
(21948, 207)
(3982, 207)


In [46]:
class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.

    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """
    def __init__(self, sigma=0.5, is_relative_detach=True):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.register_buffer('noise', torch.tensor(0))

    def forward(self, x):
        if self.training and self.sigma != 0:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.expand(*x.size()).float().normal_() * scale
            x = x + sampled_noise
        return x 


In [47]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [48]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

In [49]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [50]:
from scipy.special import expit, logit

y = train_targets_scored.iloc[:, 1:].values.astype('float32')
output_bias = torch.from_numpy(logit(y.mean(0))).cuda()

In [51]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, hidden_size)
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2619422201258426)
        self.dense2 = nn.Linear(hidden_size, hidden_size)
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.2619422201258426)
        self.dense3 = nn.Linear(hidden_size, num_targets)
        self.gaussian = GaussianNoise()
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        x = self.gaussian(x)
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x) 
        #print(output_bias)
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [52]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [53]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['fold','sig_id','drug_id']]
len(feature_cols)

1041

In [33]:
dump(target_cols, '1838_5fold_target_cols_sub.bin')
dump(feature_cols, '1838_5fold_feature_cols_sub.bin')

['1838_5fold_feature_cols_sub.bin']

In [35]:
feature_cols = load('1838_7fold_feature_cols.bin')

In [54]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 50
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5           
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=1500

In [35]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['fold'] != fold].index
    val_idx = train[train['fold'] == fold].index
    
    train_df = train[train['fold'] != fold].reset_index(drop=True)
    valid_df = train[train['fold'] == fold].reset_index(drop=True)
    
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
   
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"D:\\MOA_Pretrained_Models\\1835_moa_submission\\FOLD{fold}_{seed}.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"D:\\MOA_Pretrained_Models\\1835_moa_submission\\FOLD{fold}_{seed}.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), len(target_cols)))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions


In [36]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [37]:
# Averaging on multiple SEEDS

SEED = [0,1,2,3,4,5,6] #<-- Update
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions

FOLD: 0, EPOCH: 0, train_loss: 0.6766013688799264
FOLD: 0, EPOCH: 0, valid_loss: 0.3061637920992715
FOLD: 0, EPOCH: 1, train_loss: 0.05545177756120329
FOLD: 0, EPOCH: 1, valid_loss: 0.021790798061660357
FOLD: 0, EPOCH: 2, train_loss: 0.023691343548505203
FOLD: 0, EPOCH: 2, valid_loss: 0.020642712605851037
FOLD: 0, EPOCH: 3, train_loss: 0.02278568135385496
FOLD: 0, EPOCH: 3, valid_loss: 0.021501189470291136
FOLD: 0, EPOCH: 4, train_loss: 0.021670935162599537
FOLD: 0, EPOCH: 4, valid_loss: 0.01925069040485791
FOLD: 0, EPOCH: 5, train_loss: 0.02017013229213763
FOLD: 0, EPOCH: 5, valid_loss: 0.018957637277032646
FOLD: 0, EPOCH: 6, train_loss: 0.019615344716694908
FOLD: 0, EPOCH: 6, valid_loss: 0.018628311183835778
FOLD: 0, EPOCH: 7, train_loss: 0.019924251173717388
FOLD: 0, EPOCH: 7, valid_loss: 0.019174408433692797
FOLD: 0, EPOCH: 8, train_loss: 0.019670880764074947
FOLD: 0, EPOCH: 8, valid_loss: 0.018842984523091997
FOLD: 0, EPOCH: 9, train_loss: 0.019349242101652897
FOLD: 0, EPOCH: 9, v

FOLD: 1, EPOCH: 29, train_loss: 0.019463858043734173
FOLD: 1, EPOCH: 29, valid_loss: 0.018327917131994453
FOLD: 1, EPOCH: 30, train_loss: 0.019363344041970525
FOLD: 1, EPOCH: 30, valid_loss: 0.018260551722986358
FOLD: 1, EPOCH: 31, train_loss: 0.019218356736058737
FOLD: 1, EPOCH: 31, valid_loss: 0.01851314173213073
FOLD: 1, EPOCH: 32, train_loss: 0.019076334013447275
FOLD: 1, EPOCH: 32, valid_loss: 0.018375047083411897
FOLD: 1, EPOCH: 33, train_loss: 0.01880918999277327
FOLD: 1, EPOCH: 33, valid_loss: 0.018520838208496572
FOLD: 1, EPOCH: 34, train_loss: 0.018643408979758296
FOLD: 1, EPOCH: 34, valid_loss: 0.01841570373092379
FOLD: 1, EPOCH: 35, train_loss: 0.018465572138772392
FOLD: 1, EPOCH: 35, valid_loss: 0.018434720113873482
FOLD: 1, EPOCH: 36, train_loss: 0.0180838421585351
FOLD: 1, EPOCH: 36, valid_loss: 0.018317454361489842
FOLD: 1, EPOCH: 37, train_loss: 0.017789483627807485
FOLD: 1, EPOCH: 37, valid_loss: 0.018435605402503694
FOLD: 1, EPOCH: 38, train_loss: 0.01743848982137920

FOLD: 3, EPOCH: 8, train_loss: 0.01980561927716801
FOLD: 3, EPOCH: 8, valid_loss: 0.01747887855505242
FOLD: 3, EPOCH: 9, train_loss: 0.019745587789710018
FOLD: 3, EPOCH: 9, valid_loss: 0.017613860531984007
FOLD: 3, EPOCH: 10, train_loss: 0.01972319830910883
FOLD: 3, EPOCH: 10, valid_loss: 0.017462148358497548
FOLD: 3, EPOCH: 11, train_loss: 0.019802537652245468
FOLD: 3, EPOCH: 11, valid_loss: 0.017667016343158835
FOLD: 3, EPOCH: 12, train_loss: 0.019914073721114277
FOLD: 3, EPOCH: 12, valid_loss: 0.01762766675914035
FOLD: 3, EPOCH: 13, train_loss: 0.019899881242409996
FOLD: 3, EPOCH: 13, valid_loss: 0.017581640140098685
FOLD: 3, EPOCH: 14, train_loss: 0.019931914865214756
FOLD: 3, EPOCH: 14, valid_loss: 0.01803025241722079
FOLD: 3, EPOCH: 15, train_loss: 0.02002073686731898
FOLD: 3, EPOCH: 15, valid_loss: 0.01769740183782928
FOLD: 3, EPOCH: 16, train_loss: 0.02001200085910766
FOLD: 3, EPOCH: 16, valid_loss: 0.017396341954522273
FOLD: 3, EPOCH: 17, train_loss: 0.020074239797026352
FOLD:

FOLD: 4, EPOCH: 37, train_loss: 0.01757574403334571
FOLD: 4, EPOCH: 37, valid_loss: 0.01795235097940479
FOLD: 4, EPOCH: 38, train_loss: 0.017031658832253754
FOLD: 4, EPOCH: 38, valid_loss: 0.017734343159411636
FOLD: 4, EPOCH: 39, train_loss: 0.01651288391482355
FOLD: 4, EPOCH: 39, valid_loss: 0.017757127966199604
FOLD: 4, EPOCH: 40, train_loss: 0.0160039731071911
FOLD: 4, EPOCH: 40, valid_loss: 0.017883742307978016
FOLD: 4, EPOCH: 41, train_loss: 0.015311354676774446
FOLD: 4, EPOCH: 41, valid_loss: 0.01802949176302978
FOLD: 4, EPOCH: 42, train_loss: 0.014540898080483295
FOLD: 4, EPOCH: 42, valid_loss: 0.018103335638131414
FOLD: 4, EPOCH: 43, train_loss: 0.013781111059791368
FOLD: 4, EPOCH: 43, valid_loss: 0.017996754789991038
FOLD: 4, EPOCH: 44, train_loss: 0.012982863777627548
FOLD: 4, EPOCH: 44, valid_loss: 0.018251048560653415
FOLD: 4, EPOCH: 45, train_loss: 0.012304670946753544
FOLD: 4, EPOCH: 45, valid_loss: 0.018181293856884752
FOLD: 4, EPOCH: 46, train_loss: 0.01162255341457068


FOLD: 1, EPOCH: 16, train_loss: 0.019848944823237232
FOLD: 1, EPOCH: 16, valid_loss: 0.01834251768887043
FOLD: 1, EPOCH: 17, train_loss: 0.01974400350429716
FOLD: 1, EPOCH: 17, valid_loss: 0.01857247150370053
FOLD: 1, EPOCH: 18, train_loss: 0.019861996812868293
FOLD: 1, EPOCH: 18, valid_loss: 0.018834499908345088
FOLD: 1, EPOCH: 19, train_loss: 0.019834494789260148
FOLD: 1, EPOCH: 19, valid_loss: 0.018707631794469697
FOLD: 1, EPOCH: 20, train_loss: 0.01988156795175406
FOLD: 1, EPOCH: 20, valid_loss: 0.018393647324826037
FOLD: 1, EPOCH: 21, train_loss: 0.019845735376877505
FOLD: 1, EPOCH: 21, valid_loss: 0.018411326142294068
FOLD: 1, EPOCH: 22, train_loss: 0.019832389918665816
FOLD: 1, EPOCH: 22, valid_loss: 0.018800001431788716
FOLD: 1, EPOCH: 23, train_loss: 0.019822951188705265
FOLD: 1, EPOCH: 23, valid_loss: 0.01850156874528953
FOLD: 1, EPOCH: 24, train_loss: 0.0197619441896677
FOLD: 1, EPOCH: 24, valid_loss: 0.018628939241170882
FOLD: 1, EPOCH: 25, train_loss: 0.01980803757362122
F

...

FOLD: 4, EPOCH: 48, train_loss: 0.012831644094346659
FOLD: 4, EPOCH: 48, valid_loss: 0.018064929199005875
FOLD: 4, EPOCH: 49, train_loss: 0.012753013298725304
FOLD: 4, EPOCH: 49, valid_loss: 0.017985009614910397
FOLD: 0, EPOCH: 0, train_loss: 0.675166663484297
FOLD: 0, EPOCH: 0, valid_loss: 0.2995499449116843
FOLD: 0, EPOCH: 1, train_loss: 0.054272661484993885
FOLD: 0, EPOCH: 1, valid_loss: 0.02139638103544712
FOLD: 0, EPOCH: 2, train_loss: 0.023486029910112637
FOLD: 0, EPOCH: 2, valid_loss: 0.023408527033669607
FOLD: 0, EPOCH: 3, train_loss: 0.02285077417458313
FOLD: 0, EPOCH: 3, valid_loss: 0.019832041966063637
FOLD: 0, EPOCH: 4, train_loss: 0.021299422556615395
FOLD: 0, EPOCH: 4, valid_loss: 0.03627440519630909
FOLD: 0, EPOCH: 5, train_loss: 0.02287986507450325
FOLD: 0, EPOCH: 5, valid_loss: 0.01945325072322573
FOLD: 0, EPOCH: 6, train_loss: 0.020445218282765236
FOLD: 0, EPOCH: 6, valid_loss: 0.018883735686540604
FOLD: 0, EPOCH: 7, train_loss: 0.02007048246383235
FOLD: 0, EPOCH: 7, 

FOLD: 1, EPOCH: 27, train_loss: 0.019651643646350744
FOLD: 1, EPOCH: 27, valid_loss: 0.018452953600457737
FOLD: 1, EPOCH: 28, train_loss: 0.01949085144285303
FOLD: 1, EPOCH: 28, valid_loss: 0.01855756076318877
FOLD: 1, EPOCH: 29, train_loss: 0.01937561376142676
FOLD: 1, EPOCH: 29, valid_loss: 0.01829811999840396
FOLD: 1, EPOCH: 30, train_loss: 0.019268990514704782
FOLD: 1, EPOCH: 30, valid_loss: 0.01849545905632632
FOLD: 1, EPOCH: 31, train_loss: 0.019183122627709034
FOLD: 1, EPOCH: 31, valid_loss: 0.018400405773094722
FOLD: 1, EPOCH: 32, train_loss: 0.019049808450967726
FOLD: 1, EPOCH: 32, valid_loss: 0.018400150431053978
FOLD: 1, EPOCH: 33, train_loss: 0.018854111772927926
FOLD: 1, EPOCH: 33, valid_loss: 0.01835743673145771
FOLD: 1, EPOCH: 34, train_loss: 0.018618911572725234
FOLD: 1, EPOCH: 34, valid_loss: 0.01814040185085365
FOLD: 1, EPOCH: 35, train_loss: 0.018400672218170915
FOLD: 1, EPOCH: 35, valid_loss: 0.018616147924746786
FOLD: 1, EPOCH: 36, train_loss: 0.018122325304650914


FOLD: 3, EPOCH: 6, train_loss: 0.019907911254120045
FOLD: 3, EPOCH: 6, valid_loss: 0.01745713658302146
FOLD: 3, EPOCH: 7, train_loss: 0.01990327922006448
FOLD: 3, EPOCH: 7, valid_loss: 0.01758488656624275
FOLD: 3, EPOCH: 8, train_loss: 0.019684335041413273
FOLD: 3, EPOCH: 8, valid_loss: 0.017679646748172885
FOLD: 3, EPOCH: 9, train_loss: 0.01957848433243192
FOLD: 3, EPOCH: 9, valid_loss: 0.017774593249401626
FOLD: 3, EPOCH: 10, train_loss: 0.019658338494490887
FOLD: 3, EPOCH: 10, valid_loss: 0.01735991403898772
FOLD: 3, EPOCH: 11, train_loss: 0.01972248498350382
FOLD: 3, EPOCH: 11, valid_loss: 0.017429772737052512
FOLD: 3, EPOCH: 12, train_loss: 0.019736933805372402
FOLD: 3, EPOCH: 12, valid_loss: 0.01747547514627085
FOLD: 3, EPOCH: 13, train_loss: 0.019845070006946724
FOLD: 3, EPOCH: 13, valid_loss: 0.0176958518979304
FOLD: 3, EPOCH: 14, train_loss: 0.019943354921280475
FOLD: 3, EPOCH: 14, valid_loss: 0.01761919301112785
FOLD: 3, EPOCH: 15, train_loss: 0.019917824319091396
FOLD: 3, EP

FOLD: 4, EPOCH: 35, train_loss: 0.018623976617295673
FOLD: 4, EPOCH: 35, valid_loss: 0.017822241969406606
FOLD: 4, EPOCH: 36, train_loss: 0.018413183697755787
FOLD: 4, EPOCH: 36, valid_loss: 0.017630799008267268
FOLD: 4, EPOCH: 37, train_loss: 0.018125555383554405
FOLD: 4, EPOCH: 37, valid_loss: 0.017942858806678227
FOLD: 4, EPOCH: 38, train_loss: 0.01769485195720757
FOLD: 4, EPOCH: 38, valid_loss: 0.017720654074634825
FOLD: 4, EPOCH: 39, train_loss: 0.017275539503527292
FOLD: 4, EPOCH: 39, valid_loss: 0.017820251653237
FOLD: 4, EPOCH: 40, train_loss: 0.01687312984596128
FOLD: 4, EPOCH: 40, valid_loss: 0.017781148398561136
FOLD: 4, EPOCH: 41, train_loss: 0.01630286142176044
FOLD: 4, EPOCH: 41, valid_loss: 0.017799791267939975
FOLD: 4, EPOCH: 42, train_loss: 0.01571030067819832
FOLD: 4, EPOCH: 42, valid_loss: 0.017827174812555312
FOLD: 4, EPOCH: 43, train_loss: 0.015036994375396465
FOLD: 4, EPOCH: 43, valid_loss: 0.017756720978234497
FOLD: 4, EPOCH: 44, train_loss: 0.014417102762862392


KeyError: "None of [Index(['5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor',\n       'acat_inhibitor', 'acetylcholine_receptor_agonist',\n       'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor',\n       'adenosine_receptor_agonist', 'adenosine_receptor_antagonist',\n       'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist',\n       ...\n       'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist',\n       'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor',\n       'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b',\n       'vitamin_d_receptor_agonist', 'wnt_inhibitor'],\n      dtype='object', length=206)] are in the [columns]"

In [75]:

import numpy as np
pred1 = np.load('moa_oof_01844.npy')
pred2 = np.load('model_oof_01840.npy')
pred3 = np.load('moa_oof_01858.npy')
pred4 = np.load('moa_oof_xgb.npy')
pred5 = np.load('model_newcv_oof_01835.npy')
pred6 = np.load('moa_oof_newcv_01844.npy')
pred7= np.load('moa_oof_newcv_01858_opt.npy')

pred8 = np.load('model_newcv_oof_01835.npy')
pred9 = np.load('moa_oof_newcv_01858_7folds.npy')
pred10 = np.load('moa_oof_newcv_01838_7folds.npy')
pred11 = np.load('moa_oof_newcv_01858.npy')



In [83]:
pred3

array([[2.45717721e-03, 1.35532549e-03, 8.05951970e-04, ...,
        1.64518212e-03, 3.17708929e-04, 1.98489587e-03],
       [2.96254625e-04, 4.14825874e-04, 1.65013833e-03, ...,
        2.28889545e-03, 3.75037021e-03, 3.05411140e-03],
       [1.71655422e-03, 1.96005346e-03, 6.99006123e-04, ...,
        1.34017773e-03, 1.94973886e-04, 1.06534110e-03],
       ...,
       [4.47388428e-04, 5.07364375e-04, 2.09519803e-03, ...,
        1.27740188e-03, 9.25638009e-03, 2.40041744e-03],
       [3.89067912e-04, 4.29120133e-04, 3.46576999e-04, ...,
        3.40592320e-04, 1.82173591e-04, 6.86126863e-04],
       [4.97435224e-04, 5.07766864e-04, 7.50074915e-04, ...,
        5.78129455e-04, 7.74010126e-05, 2.08420327e-04]])

In [86]:
train[target_cols] = (pred8 * 5 + pred11 * 2 + pred9)/8

In [87]:
pred8

array([[0.00178458, 0.00145417, 0.00095003, ..., 0.00175713, 0.00058458,
        0.00162759],
       [0.00127926, 0.00110549, 0.00242534, ..., 0.00381091, 0.00153891,
        0.0031033 ],
       [0.00179724, 0.00240839, 0.00141167, ..., 0.00124529, 0.00061229,
        0.00223328],
       ...,
       [0.00071043, 0.00117841, 0.00432172, ..., 0.00228528, 0.00329333,
        0.0029959 ],
       [0.0007111 , 0.00068154, 0.00064038, ..., 0.00043908, 0.00069466,
        0.00098637],
       [0.00054111, 0.0010432 , 0.00138893, ..., 0.00097929, 0.00055558,
        0.00106453]])

In [88]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)



y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)

CV log_loss:  0.015510270095556066


In [ ]:
def log_loss_metric(y_true, y_pred):
    metrics = []
    for _target in train_targets_scored.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels = [0,1]))
    return np.mean(metrics)

In [38]:
for i in target_cols:
    test[i]=0.

In [18]:
pred1 = np.load('moa_01844.npy')
pred2 = np.load('model_newcv_01835.npy')
pred3 = np.load('moa_01844.npy')
pred4 = np.load('model_01840.npy')
pred5 = np.load('moa_01858.npy')
pred6 = np.load('moa_xgb.npy')
pred7 = np.load('moa_newcv_01844.npy')
#np.save('moa_oof_newcv_01858',oof)

pred8 = np.load('moa_newcv_01858.npy')

pred9 = np.load('model_newcv_01835_7folds.npy')
pred10 = np.load('model_newcv_01835.npy')
pred11 = np.load('moa_newcv_01858_7folds.npy')

pred11 = np.load('moa_oof_newcv_01858_label.npy')


In [79]:
np.save('moa_oof_newcv_01838_7folds',oof)
np.save('moa_newcv_01838_7folds',predictions)

In [72]:
pred9

array([[1.10278240e-03, 6.82594264e-04, 1.19356830e-03, ...,
        1.54084741e-03, 2.05626534e-04, 1.37703803e-03],
       [4.42528811e-04, 5.94025032e-04, 1.21270533e-03, ...,
        2.35788353e-03, 1.52770092e-03, 3.52004101e-03],
       [1.20766341e-03, 1.49467286e-03, 6.83186440e-04, ...,
        3.10085644e-04, 2.16352449e-04, 8.38998438e-04],
       ...,
       [6.28481503e-04, 8.00978810e-04, 2.28777167e-03, ...,
        2.62276130e-03, 2.43461723e-03, 3.83292825e-03],
       [2.17310975e-04, 1.92161797e-04, 4.54758270e-04, ...,
        3.91382387e-04, 2.26230154e-04, 4.60107481e-04],
       [4.39636961e-04, 7.01002098e-04, 5.53188666e-04, ...,
        4.54896814e-04, 9.08720964e-05, 3.88511750e-04]])

In [39]:
test[target_cols]=predictions

In [40]:
predictions

array([[0.00115784, 0.00141058, 0.00380903, ..., 0.00219696, 0.01388766,
        0.00216653],
       [0.00088531, 0.00112943, 0.00178357, ..., 0.0016953 , 0.00121521,
        0.00259553],
       [0.00137658, 0.00127005, 0.00187087, ..., 0.00196184, 0.00109692,
        0.00234351],
       ...,
       [0.00169075, 0.00122805, 0.00113446, ..., 0.00157153, 0.00063667,
        0.00161928],
       [0.00223496, 0.00143427, 0.00143444, ..., 0.00240236, 0.00094376,
        0.00353015],
       [0.00090402, 0.0012361 , 0.00144127, ..., 0.00230748, 0.00088142,
        0.00193124]])

In [41]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission_25_11_2020.csv', index=False)

In [42]:
sub

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.001158,0.001411,0.003809,0.016010,0.014587,0.003848,0.002600,0.006484,0.000710,...,0.001353,0.001472,0.006305,0.001744,0.000909,0.001119,0.000851,0.002197,0.013888,0.002167
1,id_001897cda,0.000885,0.001129,0.001784,0.002563,0.001704,0.001912,0.007053,0.006694,0.007641,...,0.001199,0.001292,0.006214,0.001119,0.011754,0.001107,0.020860,0.001695,0.001215,0.002596
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.001377,0.001270,0.001871,0.011482,0.019169,0.005683,0.003351,0.004011,0.000631,...,0.001053,0.001324,0.003028,0.022603,0.004680,0.001113,0.002146,0.001962,0.001097,0.002344
4,id_0027f1083,0.002491,0.001491,0.002126,0.019063,0.017298,0.003556,0.005589,0.002137,0.000955,...,0.001430,0.000966,0.003337,0.002487,0.002143,0.001215,0.001660,0.001963,0.000591,0.002316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,0.001151,0.001213,0.001232,0.001703,0.006136,0.002078,0.000898,0.001514,0.000527,...,0.000830,0.004818,0.001818,0.146953,0.006884,0.001322,0.013573,0.001566,0.000777,0.001113
3978,id_ff925dd0d,0.004390,0.002331,0.001147,0.007570,0.021460,0.006598,0.005212,0.003664,0.001059,...,0.000928,0.001362,0.003895,0.003344,0.003401,0.001317,0.003540,0.001936,0.000741,0.001875
3979,id_ffb710450,0.001691,0.001228,0.001134,0.012340,0.035949,0.006084,0.003588,0.003468,0.000538,...,0.000970,0.000825,0.003330,0.001697,0.001604,0.000919,0.001054,0.001572,0.000637,0.001619
3980,id_ffbb869f2,0.002235,0.001434,0.001434,0.029621,0.028549,0.006522,0.006328,0.003200,0.000787,...,0.001060,0.000859,0.002755,0.002374,0.002036,0.001006,0.001587,0.002402,0.000944,0.003530


In [45]:
np.save('model_newcv_01838_5folds', predictions)

In [46]:
np.save('model_newcv_oof_01838_5folds', oof)

In [60]:
oof.shape

(21948, 206)